#      Content Classification using UN Content Classification Scheme

# Import Libraries & Load Data

In [70]:
#!/usr/bin/python3

#import nltk
#nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')
import os, pickle, re, string
from string import punctuation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [45]:
un_news = pd.read_csv("../Datasets/UN_News_no_NA_Dups.csv")
uni_news = pd.read_csv("../Datasets/University_News_no_NA_Dups.csv")
arabic_news = pd.read_csv("../Datasets/Arabic_News_AL-Mashareq_no_Na_Dups.csv")
#arabic_news["Category"].replace({"أمن": "السلم والأمن"}, inplace=True)
print('UN News', '\t', len(un_news), '\n\n', un_news['Category'].value_counts(), '\n\n\n',
      'University News', '\t', len(uni_news), '\n\n', uni_news['Category'].value_counts(), '\n\n\n',
      'Arabic News', '\t', len(arabic_news), '\n\n', arabic_news['Category'].value_counts())

UN News 	 5682 

 السلم والأمن               1897
حقوق الإنسان                796
الصحة                       615
المساعدات الإنسانية         510
المهاجرون واللاجئون         401
شؤون الأمم المتحدة          318
أهداف التنمية المستدامة     236
تغير المناخ                 233
المرأة                      204
الثقافة والتعليم            198
التنمية الاقتصادية          184
القانون ومنع الجريمة         90
Name: Category, dtype: int64 


 University News 	 2725 

 شراكة مجنمعية      417
مجتمع الجامعة      411
الجامعة            383
طلاب               360
دولى               238
فعاليات            177
ابحاث ومشاريع      149
قصص مميزة          136
ثقافة وفنون        119
علوم وتكنولوجيا     72
زيارات              66
جوائز ومنح          66
مؤتمرات             65
خريجون              46
رياضة               20
Name: Category, dtype: int64 


 Arabic News 	 1633 

 أمن             580
إرهاب           375
سياسة           256
حقوق الإنسان    129
صحة             100
مجتمع            64
عدالة            59

# Data Preprocessing

In [51]:
"""
** Remove English and Arabic punctuations
** Normalize different forms of letters using one common form
"""
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
  
un_news['Content'] = un_news['Content'].apply(preprocess)
un_news.head(5)

                                               Title  \
0  منظمة العمل الدولية: العاملون من المنزل بحاجة ...   
1  الأمم المتحدة: منع حدوث المجاعة في اليمن أولوي...   
2  كورونا: علماء يسعون لفهم تحور الفيروس ويحذرون ...   
3  أزمة غذاء طاحنة وأطفال يضطرون إلى التسول بالشو...   
4  مع دخول الجائحة عامها الثاني، اليونيسف تدعو إل...   

                                             Content  \
0  جاء تقرير جديد لمنظمه العمل الدوليه اوضح ان او...   
1  وقال ستيفان دوجاريك المءتمر الصحفي اليومي الثل...   
2  وقد اجتمع الخبراء منتدي افتراضي استضافته منظمه...   
3  وحذر برنامج الاغذيه العالمي التابع للامم المتح...   
4  وقالت دخولنا العام الثاني لجاءحه كوفيد19 ومع ا...   

                                           Link             Category  
0  https://news.un.org/ar/story/2021/01/1068972         حقوق الإنسان  
1  https://news.un.org/ar/story/2021/01/1068982  المساعدات الإنسانية  
2  https://news.un.org/ar/story/2021/01/1068992                الصحة  
3  https://news.un.org/ar/story/2021/01/10

In [52]:
# splitting the data into target and feature
feature = un_news.Content
target = un_news.Category

# splitting into train and tests
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

# make pipeline
pipe = make_pipeline(TfidfVectorizer(),
                    LogisticRegression())
# make param grid
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

# create and fit the model
model = GridSearchCV(pipe, param_grid, cv=5)
model.fit(X_train,Y_train)

# make prediction and print accuracy
prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

/home/asmaa/anaconda3/envs/VMK-mer/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/asmaa/anaconda3/envs/VMK-mer/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

Accuracy score is 0.74
                         precision    recall  f1-score   support

أهداف التنمية المستدامة       0.56      0.48      0.51        42
     التنمية الاقتصادية       0.44      0.48      0.46        31
       الثقافة والتعليم       0.59      0.45      0.51        29
           السلم والأمن       0.78      0.87      0.82       386
                  الصحة       0.82      0.86      0.84       128
   القانون ومنع الجريمة       0.80      0.20      0.32        20
                 المرأة       0.62      0.63      0.62        38
    المساعدات الإنسانية       0.70      0.61      0.65        98
    المهاجرون واللاجئون       0.79      0.72      0.75        92
            تغير المناخ       0.84      0.76      0.80        42
           حقوق الإنسان       0.75      0.81      0.78       162
     شؤون الأمم المتحدة       0.64      0.49      0.56        69

               accuracy                           0.74      1137
              macro avg       0.69      0.61      0.64      1137


In [55]:
pipe = make_pipeline(TfidfVectorizer(),
                    RandomForestClassifier())

param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
              'randomforestclassifier__max_features':['sqrt', 'log2']}

rf_model = GridSearchCV(pipe, param_grid, cv=5)
rf_model.fit(X_train,Y_train)

prediction = rf_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")

Accuracy score is 0.63


In [56]:
pipe = make_pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
                     ])
pipe.fit(X_train,Y_train)
prediction = pipe.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.40
                         precision    recall  f1-score   support

أهداف التنمية المستدامة       0.00      0.00      0.00        42
     التنمية الاقتصادية       0.00      0.00      0.00        31
       الثقافة والتعليم       0.00      0.00      0.00        29
           السلم والأمن       0.36      1.00      0.53       386
                  الصحة       1.00      0.31      0.48       128
   القانون ومنع الجريمة       0.00      0.00      0.00        20
                 المرأة       0.00      0.00      0.00        38
    المساعدات الإنسانية       0.00      0.00      0.00        98
    المهاجرون واللاجئون       0.00      0.00      0.00        92
            تغير المناخ       0.00      0.00      0.00        42
           حقوق الإنسان       0.93      0.17      0.28       162
     شؤون الأمم المتحدة       0.00      0.00      0.00        69

               accuracy                           0.40      1137
              macro avg       0.19      0.12      0.11      1137


/home/asmaa/anaconda3/envs/VMK-mer/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
pipe = make_pipeline(TfidfVectorizer(),
                     SVC())
param_grid = {'svc__kernel': ['rbf', 'linear', 'poly'],
             'svc__gamma': [0.1, 1, 10, 100],
             'svc__C': [0.1, 1, 10, 100]}

svc_model = GridSearchCV(pipe, param_grid, cv=3)
svc_model.fit(X_train, Y_train)

prediction = svc_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

KeyboardInterrupt: 

In [68]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, Y_train)

#%%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, Y_test))
print(classification_report(Y_test, y_pred))

accuracy 0.7115215479331575
                         precision    recall  f1-score   support

أهداف التنمية المستدامة       0.65      0.31      0.42        42
     التنمية الاقتصادية       0.60      0.39      0.47        31
       الثقافة والتعليم       0.68      0.45      0.54        29
           السلم والأمن       0.70      0.92      0.79       386
                  الصحة       0.74      0.90      0.81       128
   القانون ومنع الجريمة       0.00      0.00      0.00        20
                 المرأة       0.62      0.55      0.58        38
    المساعدات الإنسانية       0.71      0.49      0.58        98
    المهاجرون واللاجئون       0.75      0.72      0.73        92
            تغير المناخ       0.69      0.79      0.73        42
           حقوق الإنسان       0.76      0.76      0.76       162
     شؤون الأمم المتحدة       0.75      0.13      0.22        69

               accuracy                           0.71      1137
              macro avg       0.64      0.53      0.55      

In [35]:
list(arabic_news['Content'].str.contains(pat = "ارهاب")).count(True)

151

In [34]:
list(arabic_news['Category']).count('إرهاب')

375

In [ ]:
#Categories
categories_dict = {
    'algeria':1, 
    'sport':2, 
    'entertainment':3,
    'society':4, 
    'world':5, 
    'religion':6, 
}

### Tools
## Farasa Arabic NLP Toolkit
# Tokenizer
farasaSegmenter = 'Tools/farasa/segmenter'

## Arabic StopWords List
stopWords = open("Tools/arabic-stop-words/list.txt").read().splitlines()

## Models directory
models = 'Models/dumps/'

## Remove Numbers and add Other punctuation
punctuation += '،؛؟”0123456789“'

class Helper():
    def __init__(self, article = False):
        self.article = article

    ##~~Pickle helpers~~#
    def getPickleContent(self, pklFile):
        with open (pklFile, 'rb') as fp:
            itemlist = pickle.load(fp)
        return itemlist

    def setPickleContent(self, fileName, itemList):
        with open(fileName+'.pkl', 'wb') as fp:
            pickle.dump(itemList, fp)
    #~~~~~~~~~~~~~~~~~~#

    #~~~ Set and get Model
    def getModel(self, name):
        model = self.getPickleContent(os.path.join(models, name+'/model_'+name+'.pkl'))
        cv = self.getPickleContent(os.path.join(models, name+'/cv_'+name+'.pkl'))
        tfidf = self.getPickleContent(os.path.join(models, name+'/tfidf_'+name+'.pkl'))
        return model, cv, tfidf

    def setModel(self, name, model, cv, tfidf):
        path = os.path.join(models, name)
        if not os.path.exists(path):
            os.mkdir(path)
        self.setPickleContent(os.path.join(models, name+'/model_'+name), model)
        self.setPickleContent(os.path.join(models, name+'/cv_'+name), cv)
        self.setPickleContent(os.path.join(models, name+'/tfidf_'+name), tfidf)
    #~~~~~~~~~~~~~~~~~~

    # Get the article content
    def getArticleContent(self, article):
        if os.path.exists(article):
            return open(article, 'r').read()

    # Drop empty lines
    def dropNline(self, article):
        if os.path.exists(article):
            content = self.getArticleContent(article)
            return re.sub(r'\n', ' ', content)

    # Get stemmed content 
    def getLemmaArticle(self, content):
        jarFarasaSegmenter = os.path.join(farasaSegmenter, 'FarasaSegmenterJar.jar')
        tmp = os.path.join(farasaSegmenter, 'tmp')
        if os.path.exists(tmp):
            os.system('rm '+tmp)
        open(tmp, 'w').write(content)
        tmpLemma = os.path.join(farasaSegmenter, 'tmpLemma')
        if os.path.exists(tmpLemma):
            os.system('rm '+tmpLemma)
        os.system('java -jar ' + jarFarasaSegmenter + ' -l true -i ' + tmp + ' -o ' + tmpLemma)
        return self.getArticleContent(tmpLemma)

    # Remove Stop words
    def getCleanArticle(self, content):
        content = ''.join(c for c in content if c not in punctuation)
        words = content.split()     
        cleandWords = [w for w in words if w not in stopWords]
        return ' '.join(cleandWords)

    # Pre-processing Pipeline, before prediction (Get article Bag of Words)
    def pipeline(self, content):
        cleanArticle = self.getCleanArticle(content)
        lemmaContent = self.getLemmaArticle(cleanArticle)
        cleanArticle = self.getCleanArticle(lemmaContent).split()
        return ' '.join(cleanArticle)

    # Main function, predict content category
    def predict(self, content):
        article = self.pipeline(content)
        model, cv, tfidf = self.getModel('sgd_94')
        vectorized = tfidf.transform(cv.transform([article]))
        predicted = model.predict(vectorized)
        keys = list(categories_dict.keys())
        values = list(categories_dict.values())
        categoryPredicted = keys[values.index(predicted[0])].upper()
        return categoryPredicted



if __name__ == '__main__':
    help = Helper()
    content = 'أمرت السلطات القطرية الأسواق والمراكز التجارية في البلاد برفع وإزالة السلع الواردة من السعودية والبحرين والإمارات ومصر في الذكرى الأولى لإعلان هذه الدول الحصار عليها.'
    category = help.predict(content)
    print(category)